In [13]:
import os, cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from PIL import Image
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import keras.optimizers
import random
import scipy
import math
import tensorflow as tf

In [14]:
imgFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\JPEGImages'
CImagesDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\CImages'
parentImgFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages'
xmlFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_XMLAnnotations\XMLAnnotations'


In [15]:
def extractClassLabels(xmlFileDir):
    os.chdir(xmlFileDir)
    xmlFileNames = [f for f in listdir(xmlFileDir) if isfile(join(xmlFileDir, f))]
    #xmlFileNames = ['3m1.xml','3m2.xml','3m3.xml','3m4.xml']
    classLabels = []
    imageNames = []
    count = 1
    for file in xmlFileNames:
        root = ET.parse(file).getroot()
        filename = root.find('filename').text
        action = root.find('object/action').text
        classLabels.append(action)
        imageNames.append(filename)
        print(str(count)+'/'+str(len(xmlFileNames)), end='\r')
        count +=1
    return np.array(classLabels), np.array(imageNames)

In [16]:
classList, allImageNames = extractClassLabels(xmlFileDir)


In [17]:
(classUnique,counts) = np.unique(classList, return_counts=True)

(_,inverse) = np.unique(classList, return_inverse=True)

y = to_categorical(inverse,len(classUnique))

In [18]:
batch_size = 32
img_height = 90
img_width = 90
actionImagesDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\ActionImages'
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    actionImagesDir,
    validation_split=0.2,
    label_mode = 'categorical',
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    actionImagesDir,
    validation_split=0.2,
    label_mode = 'categorical',
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 9532 files belonging to 40 classes.
Using 7626 files for training.
Found 9532 files belonging to 40 classes.
Using 1906 files for validation.


In [19]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [20]:
#AUTOTUNE = tf.data.experimental.AUTOTUNE
#train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [21]:
def define_model(nrOfClasses, hyperparameters):
    model = Sequential()
    
    model.add(Conv2D(hyperparameters["nrOfFiltersInConvLayers"][0], (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(90 ,90, 3)))
    
    for nrOfFilters in hyperparameters["nrOfFiltersInConvLayers"][1:]:
        model.add(Conv2D(nrOfFilters, (3, 3), activation='relu', kernel_initializer='he_uniform'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(hyperparameters["dropout"]))

    model.add(Flatten())
    
    for nrOfNeurons in hyperparameters["nrOfNeuronsInHiddenLayers"]:
        model.add(Dense(nrOfNeurons, activation='relu', kernel_initializer='he_uniform'))
        
    model.add(Dense(nrOfClasses, activation='softmax'))
    # compile model¨
    opt = keras.optimizers.Adam(learning_rate=hyperparameters["learningRate"])
    #opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    #model.summary()
    return model

In [22]:
nrOfConvLayers = [2,3,4]
nrOfFilters = [16,32,64,128]
nrOfHiddenLayers = [2,3,4]
nrOfHiddenLayerNeurons = [32,64,128,256,512]
dropout = [0.2,0.5,0.8]
learningRate = [0.00001,0.0001,0.001]

tests = 100

bestAccuracy = 0
bestHyperparameters = None

for test in range(tests):
    hyperparameters = {
        'nrOfFiltersInConvLayers': [],
        'nrOfNeuronsInHiddenLayers':[],
        'dropout': random.choice(dropout),
        'learningRate':random.choice(learningRate)
    }
    
    for layer in range(random.choice(nrOfConvLayers)):
        hyperparameters['nrOfFiltersInConvLayers'].append(random.choice(nrOfFilters))
    
    
    for layer in range(random.choice(nrOfHiddenLayers)):
        hyperparameters['nrOfNeuronsInHiddenLayers'].append(random.choice(nrOfHiddenLayerNeurons))
                       
    print(hyperparameters)
    
    model = define_model(len(classUnique),hyperparameters)
    model.fit(train_ds, epochs=20)
    result = model.evaluate(test_ds, batch_size=32)
    accuracy = result[1]
    
    if accuracy > bestAccuracy:
        bestAccuracy = accuracy
        bestHyperparameters = hyperparameters
    
    print(bestAccuracy)
    print(bestHyperparameters)

{'nrOfFiltersInConvLayers': [32, 16, 32, 32], 'nrOfNeuronsInHiddenLayers': [64, 64, 512, 32], 'dropout': 0.8, 'learningRate': 0.001}
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
239/239 [==============================] - 10s 40ms/step - loss: 3.9036 - accuracy: 0.0298
Epoch 2/20
239/239 [==============================] - 11s 47ms/step - loss: 3.6834 - accuracy: 0.0317
Epoch 3/20
239/239 [=====================

60/60 [==============================] - 2s 30ms/step - loss: 525.2593 - accuracy: 0.1679
0.16789087653160095
{'nrOfFiltersInConvLayers': [16, 64, 64], 'nrOfNeuronsInHiddenLayers': [128, 128], 'dropout': 0.2, 'learningRate': 0.0001}
{'nrOfFiltersInConvLayers': [16, 64, 128], 'nrOfNeuronsInHiddenLayers': [32, 32, 512, 32], 'dropout': 0.2, 'learningRate': 1e-05}
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
239/

60/60 [==============================] - 2s 26ms/step - loss: 3.6821 - accuracy: 0.0252
0.16789087653160095
{'nrOfFiltersInConvLayers': [16, 64, 64], 'nrOfNeuronsInHiddenLayers': [128, 128], 'dropout': 0.2, 'learningRate': 0.0001}
{'nrOfFiltersInConvLayers': [16, 64, 16], 'nrOfNeuronsInHiddenLayers': [64, 128, 32], 'dropout': 0.8, 'learningRate': 1e-05}
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
239/239 [==

60/60 [==============================] - 2s 30ms/step - loss: 3.6802 - accuracy: 0.0252 0s - loss: 3.6793 - accu - ETA: 0s - loss: 3.6796 - accuracy: 
0.16789087653160095
{'nrOfFiltersInConvLayers': [16, 64, 64], 'nrOfNeuronsInHiddenLayers': [128, 128], 'dropout': 0.2, 'learningRate': 0.0001}
{'nrOfFiltersInConvLayers': [128, 64, 128, 16], 'nrOfNeuronsInHiddenLayers': [32, 64, 64, 64], 'dropout': 0.5, 'learningRate': 0.001}
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
60/60 [==============================] - 2s 35ms/step - loss: 123.8756 - accuracy: 0.0567
0.16789087653160095
{'nrOfFiltersInConvLayers': [16, 64, 64], 'nrOfNeuronsInHiddenLayers': [128, 128], 'dropout': 0.2, 'learningRate': 0.0001}
{'nrOfFiltersInConvLayers': [16, 32, 64, 16], 'nrOfNeuronsInHiddenLayers': [32, 256, 64], 'dropout': 0.2, 'learningRate': 0.001}
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Pleas

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
239/239 [==============================] - 13s 54ms/step - loss: 4.4896 - accuracy: 0.0307
Epoch 2/20
239/239 [==============================] - 14s 58ms/step - loss: 3.6784 - accuracy: 0.0309
Epoch 3/20
239/239 [==============================] - 14s 57ms/step - loss: 3.6781 - accuracy: 0.0315
Epoch 4/20
239/239 [==============================] - 13s 56ms/step - loss: 3.6781 - accuracy: 0.0309
Epoch 5/20
239/239 [==============================] - 14s 58ms/step - loss: 3.6777 - accuracy: 0.0316
Epoch 6/20
239/239 [==============================] - 14s 57ms/step - loss: 3.6777 - accuracy: 0.0316
Epoch 7/20
239/239 [==============================] - 13s 56ms/step - loss: 3.6776 - acc

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
239/239 [==============================] - 28s 117ms/step - loss: 3.7682 - accuracy: 0.0290
Epoch 2/20
239/239 [==============================] - 25s 107ms/step - loss: 3.6880 - accuracy: 0.0354
Epoch 3/20
239/239 [==============================] - 25s 105ms/step - loss: 3.6809 - accuracy: 0.0374
Epoch 4/20
239/239 [==============================] - 25s 106ms/step 

60/60 [==============================] - 2s 32ms/step - loss: 1710.3024 - accuracy: 0.1406
0.21301154792308807
{'nrOfFiltersInConvLayers': [32, 32, 128, 128], 'nrOfNeuronsInHiddenLayers': [256, 256], 'dropout': 0.2, 'learningRate': 0.0001}
{'nrOfFiltersInConvLayers': [128, 32], 'nrOfNeuronsInHiddenLayers': [32, 64], 'dropout': 0.5, 'learningRate': 1e-05}
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
239/239 [=

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
60/60 [==============================] - 2s 32ms/step - loss: 3.6803 - accuracy: 0.0252
0.21301154792308807
{'nrOfFiltersInConvLayers': [32, 32, 128, 128], 'nrOfNeuronsInHiddenLayers': [256, 256], 'dropout': 0.2, 'learningRate': 0.0001}
{'nrOfFiltersInConvLayers': [32, 32], 'nrOfNeuronsInHiddenLayers': [512, 32], 'dropout': 0.2, 'learningRate': 0.001}
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report

239/239 [==============================] - 10s 41ms/step - loss: 3.5760 - accuracy: 0.0563
Epoch 16/20
239/239 [==============================] - 10s 40ms/step - loss: 3.5573 - accuracy: 0.0623
Epoch 17/20
239/239 [==============================] - 10s 40ms/step - loss: 3.5469 - accuracy: 0.0644
Epoch 18/20
239/239 [==============================] - 10s 40ms/step - loss: 3.5366 - accuracy: 0.0669
Epoch 19/20
239/239 [==============================] - 10s 41ms/step - loss: 3.5120 - accuracy: 0.0675
Epoch 20/20
239/239 [==============================] - 10s 41ms/step - loss: 3.4913 - accuracy: 0.07211s - l
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbo